# SQL Relational Data Analysis 


## Who Got Snubbed?

**This analysis looks at historical baseball data to determine who throughout the years got snubbed from a Hall of Fame admission.** 

## Import Libaries 

In [8]:
import numpy as np
import pandas as pd
import sqlite3
import os
import sqlite3

## Getting started

Data - [Lahman's baseball database](http://www.seanlahman.com/baseball-archive/statistics/)  

In [9]:
con = sqlite3.connect("lahmansbaseballdb.sqlite")
# get the db name
db_name = pd.read_sql("PRAGMA database_list;", con)["name"][0]

In [10]:
# Grabbing all tables in the database

list_tables = "SELECT * FROM {}.sqlite_master WHERE type='table';".format(db_name)

lahmans = pd.read_sql(list_tables , con=con)

# lahmans[0:5]
lahmans

,type,name,tbl_name,rootpage,sql
0,table,allstarfull,allstarfull,2,"CREATE TABLE ""allstarfull"" (\n\t""ID"" INTEGER N..."
1,table,appearances,appearances,151,"CREATE TABLE ""appearances"" (\n\t""ID"" INTEGER N..."
2,table,awardsmanagers,awardsmanagers,3436,"CREATE TABLE ""awardsmanagers"" (\n\t""ID"" INTEGE..."
3,table,awardsplayers,awardsplayers,3445,"CREATE TABLE ""awardsplayers"" (\n\t""ID"" INTEGER..."
4,table,awardssharemanagers,awardssharemanagers,3603,"CREATE TABLE ""awardssharemanagers"" (\n\t""ID"" I..."
5,table,awardsshareplayers,awardsshareplayers,3622,"CREATE TABLE ""awardsshareplayers"" (\n\t""ID"" IN..."
6,table,batting,batting,3757,"CREATE TABLE ""batting"" (\n\t""ID"" INTEGER NOT N..."
7,table,battingpost,battingpost,6554,"CREATE TABLE ""battingpost"" (\n\t""ID"" INTEGER N..."
8,table,collegeplaying,collegeplaying,7006,"CREATE TABLE ""collegeplaying"" (\n\t""ID"" INTEGE..."
9,table,divisions,divisions,7282,"CREATE TABLE ""divisions"" (\n\t""ID"" INTEGER NOT..."


In [11]:
# data is only current to 2018! Worth noting because some players may still be in the league and could end up in the HOF
pd.read_sql_query('select max(yearid) from halloffame', con) 

,max(yearid)
0,2018


In [12]:
rowcount = 5
my_sql_statement = '''SELECT * FROM halloffame LIMIT 5''' 
# Selecting everything(*) from table halloffame limited to first 5 rows


pd.read_sql_query(my_sql_statement, con)

,ID,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,1,cobbty01,1936,BBWAA,226,170,222,Y,Player,None
1,2,ruthba01,1936,BBWAA,226,170,215,Y,Player,None
2,3,wagneho01,1936,BBWAA,226,170,215,Y,Player,None
3,4,mathech01,1936,BBWAA,226,170,205,Y,Player,None
4,5,johnswa01,1936,BBWAA,226,170,189,Y,Player,None


What do the column fields represent?

    Fields: ID	playerID	yearid	votedBy	ballots	needed	votes	inducted	category	needed_note.
**I think these represent the features that  are taken into consideration in determining of players should be added to the hall of fame.**

In [13]:
list_from_pawards_table = '''SELECT * FROM awardsplayers LIMIT 5''' # Select everything from awardsplayer table
# both awardsplayers and hall of fame have a playerid field
pd.read_sql_query(list_from_pawards_table, con)

,ID,playerID,awardID,yearID,lgID,tie,notes
0,1,bondto01,Pitching Triple Crown,1877,NL,None,None
1,2,hinespa01,Triple Crown,1878,NL,None,None
2,3,heckegu01,Pitching Triple Crown,1884,AA,None,None
3,4,radboch01,Pitching Triple Crown,1884,NL,None,None
4,5,oneilti01,Triple Crown,1887,AA,None,None


In [14]:
# creating a new table that combines awardsplayers and halloffame
SQLjoin = '''SELECT * FROM halloffame INNER JOIN awardsplayers on
            awardsplayers.playerid=halloffame.playerid'''
pd.read_sql_query(SQLjoin, con)


,ID,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note,ID,playerID,awardID,yearID,lgID,tie,notes
0,1,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None,26,cobbty01,Baseball Magazine All-Star,1908,AL,None,RF
1,1,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None,40,cobbty01,Baseball Magazine All-Star,1908,ML,None,RF
2,1,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None,70,cobbty01,Baseball Magazine All-Star,1909,AL,None,RF
3,1,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None,86,cobbty01,Baseball Magazine All-Star,1909,ML,None,RF
4,1,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None,114,cobbty01,Baseball Magazine All-Star,1910,AL,None,RF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23125,4191,trammal01,2018,Veterans,NaN,NaN,NaN,Y,Player,None,4206,trammal01,Silver Slugger,1990,AL,None,SS
23126,4191,trammal01,2018,Veterans,NaN,NaN,NaN,Y,Player,None,3984,trammal01,TSN All-Star,1987,AL,None,SS
23127,4191,trammal01,2018,Veterans,NaN,NaN,NaN,Y,Player,None,4065,trammal01,TSN All-Star,1988,AL,None,SS
23128,4191,trammal01,2018,Veterans,NaN,NaN,NaN,Y,Player,None,4226,trammal01,TSN All-Star,1990,AL,None,SS


In [15]:
pd.read_sql_query('select * from allstarfull limit 5', con)

,ID,playerID,yearID,gameNum,gameID,teamID,team_ID,lgID,GP,startingPos
0,1,gomezle01,1933,0,ALS193307060,NYA,921,AL,1,1
1,2,ferreri01,1933,0,ALS193307060,BOS,912,AL,1,2
2,3,gehrilo01,1933,0,ALS193307060,NYA,921,AL,1,3
3,4,gehrich01,1933,0,ALS193307060,DET,919,AL,1,4
4,5,dykesji01,1933,0,ALS193307060,CHA,915,AL,1,5


To investigate the question, I  need to find out how many all star games each player played in.


In [16]:
pd.read_sql_query('select distinct gameNum from allstarfull', con)

,gameNum
0,0
1,1
2,2


Based on your investigation, what does `gameNum` describe in the `allstarfull` table?

**According to readme of the dataset, gameNum is "Game number (zero if only one All-Star game played that season)". This is the number of All-star games played by all of the players that have played in at least 1 all star game (Players with 0 have played in at least one all star game)**

In [17]:
your_query = '''SELECT allstarfull.playerID, nameLast, nameFirst, gameID FROM allstarfull INNER JOIN people on
            allstarfull.playerid=people.playerid'''
all_stars = pd.read_sql_query(your_query, con)
all_stars

,playerID,nameLast,nameFirst,gameID
0,aaronha01,Aaron,Hank,NLS195507120
1,aaronha01,Aaron,Hank,ALS195607100
2,aaronha01,Aaron,Hank,NLS195707090
3,aaronha01,Aaron,Hank,ALS195807080
4,aaronha01,Aaron,Hank,NLS195907070
...,...,...,...,...
5337,zitoba01,Zito,Barry,ALS200307150
5338,zitoba01,Zito,Barry,NLS200607110
5339,zobribe01,Zobrist,Ben,NLS200907140
5340,zobribe01,Zobrist,Ben,NLS201307160




Querying the `people` table to investigate more about `aaronha01`. 

In [18]:

your_query = '''SELECT * FROM allstarfull WHERE playerid="aaronha01"'''
rw = pd.read_sql_query(your_query, con)
rw

,ID,playerID,yearID,gameNum,gameID,teamID,team_ID,lgID,GP,startingPos
0,1130,aaronha01,1955,0,NLS195507120,ML1,1273,NL,1,NaN
1,1182,aaronha01,1956,0,ALS195607100,ML1,1289,NL,1,NaN
2,1232,aaronha01,1957,0,NLS195707090,ML1,1305,NL,1,9.0
3,1282,aaronha01,1958,0,ALS195807080,ML1,1321,NL,1,9.0
4,1373,aaronha01,1959,1,NLS195907070,ML1,1337,NL,1,9.0
5,1374,aaronha01,1959,2,NLS195908030,ML1,1337,NL,1,9.0
6,1491,aaronha01,1960,1,ALS196007110,ML1,1353,NL,1,9.0
7,1492,aaronha01,1960,2,ALS196007130,ML1,1353,NL,1,9.0
8,1607,aaronha01,1961,1,NLS196107110,ML1,1371,NL,1,NaN
9,1608,aaronha01,1961,2,ALS196107310,ML1,1371,NL,1,NaN




**Aaron hank had played 23 seasons in the MLB and and is widely regarded as one of the greatest baseball players in history. According to wikipedia, "His 755 career home runs broke the long-standing MLB record set by Babe Ruth and stood as the most for 33 years.**

In [29]:
g = all_stars.groupby('playerID').size() # line 1
# print(g)
all_star_counts = g.reset_index() # line 2
# print(all_star_counts)
all_star_counts = all_star_counts.rename(columns={0: "N_all_star_games"}) # line 3
# all_star_counts

Sorting `all_star_counts` on the field `N_all_star_games`. Which players played in the most all star games?

In [30]:
all_star_counts.sort_values(by=['N_all_star_games'], inplace=True, ascending=False)
# all_star_counts
your_query2 = '''SELECT allstarfull.playerID, nameLast, nameFirst FROM allstarfull INNER JOIN people on
            allstarfull.playerid=people.playerid'''
names = pd.read_sql_query(your_query2, con)
# names

most_games = pd.merge(all_star_counts, names, on='playerID')
# most_games
most_games = most_games.drop_duplicates(subset=None, keep='first', inplace=False) # https://www.statology.org/pandas-drop-duplicates/
most_games

,playerID,N_all_star_games,nameLast,nameFirst
0,aaronha01,24,Aaron,Hank
24,mayswi01,24,Mays,Willie
48,musiast01,24,Musial,Stan
72,mantlmi01,19,Mantle,Mickey
91,ripkeca01,19,Ripken,Cal
...,...,...,...,...
5337,millesh01,1,Miller,Shelby
5338,millwke01,1,Millwood,Kevin
5339,milnaal01,1,Milnar,Al
5340,miltoer01,1,Milton,Eric


## Merging hall of fame data


In [31]:
hall = pd.read_sql_query('select  * from halloffame', con)
hall


,ID,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,1,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None
1,2,ruthba01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
2,3,wagneho01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
3,4,mathech01,1936,BBWAA,226.0,170.0,205.0,Y,Player,None
4,5,johnswa01,1936,BBWAA,226.0,170.0,189.0,Y,Player,None
...,...,...,...,...,...,...,...,...,...,...
4186,4187,lidgebr01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4187,4188,millwke01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4188,4189,zambrca01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4189,4190,morrija02,2018,Veterans,NaN,NaN,NaN,Y,Player,None




Is each player listed one time in the halloffame table? 

In [32]:
# Your code and answer here
hall["playerID"].value_counts()

roushed01    20
youngro01    19
ruffire01    18
schalra01    18
vanceda01    17
             ..
fregoji01     1
boutoji01     1
johnsda02     1
stanlmi01     1
zambrca01     1
Name: playerID, Length: 1279, dtype: int64

**There are multiple players in the hall of fame table that are listed more than once. The highest number of occurance
is playerid roushed01**

## Deliverable 11

How many times is the player `vanceda01` listed in the `halloffame` table? Why so many times?

**Vanceda01 is listed 17 times in the halloffame table. From the code below you can see that it took 17 tries before this player received the votes needed to be inducted. you can also see this under the inducted column, he is not accepted until his 17th time.**

In [23]:
player = pd.read_sql_query('select  * from halloffame WHERE playerid="vanceda01"', con)
player

,ID,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,104,vanceda01,1936,BBWAA,226,170.0,1,N,Player,None
1,146,vanceda01,1937,BBWAA,201,151.0,10,N,Player,None
2,265,vanceda01,1938,BBWAA,262,197.0,10,N,Player,None
3,381,vanceda01,1939,BBWAA,274,206.0,15,N,Player,None
4,492,vanceda01,1942,BBWAA,233,175.0,37,N,Player,None
5,569,vanceda01,1945,BBWAA,247,186.0,18,N,Player,None
6,686,vanceda01,1946,Nominating Vote,202,NaN,31,N,Player,Top 20
7,764,vanceda01,1947,BBWAA,161,121.0,50,N,Player,None
8,804,vanceda01,1948,BBWAA,121,91.0,23,N,Player,None
9,921,vanceda01,1949,BBWAA,153,115.0,33,N,Player,None


Adjusting the `hall_sql_query` only select rows from the `halloffame` table where inducted is "Y".

In [33]:
inducted_query = '''select * from halloffame WHERE inducted = "Y"'''
hall = pd.read_sql_query(inducted_query, con)
hall

,ID,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,1,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None
1,2,ruthba01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
2,3,wagneho01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
3,4,mathech01,1936,BBWAA,226.0,170.0,205.0,Y,Player,None
4,5,johnswa01,1936,BBWAA,226.0,170.0,189.0,Y,Player,None
...,...,...,...,...,...,...,...,...,...,...
318,4158,guerrvl01,2018,BBWAA,422.0,317.0,392.0,Y,Player,None
319,4159,thomeji01,2018,BBWAA,422.0,317.0,379.0,Y,Player,None
320,4160,hoffmtr01,2018,BBWAA,422.0,317.0,337.0,Y,Player,None
321,4190,morrija02,2018,Veterans,NaN,NaN,NaN,Y,Player,None


Merging the `hall` dataframe with the `all_star_counts` dataframe. 

In [34]:

merged = pd.merge(all_star_counts, hall, how='left', on='playerID')
all_stars_vs_hall = merged[['playerID', "N_all_star_games", "inducted"]]
all_stars_vs_hall = all_stars_vs_hall.fillna(value="N")
all_stars_vs_hall

,playerID,N_all_star_games,inducted
0,aaronha01,24,Y
1,mayswi01,24,Y
2,musiast01,24,Y
3,mantlmi01,19,Y
4,ripkeca01,19,Y
...,...,...,...
1862,millesh01,1,N
1863,millwke01,1,N
1864,milnaal01,1,N
1865,miltoer01,1,N



Sorting the `all_stars_vs_hall` table by `N_all_star_games` in descending order.

In [26]:
all_stars_vs_hall.sort_values(by=['N_all_star_games'], inplace=True, ascending=False)
all_stars_vs_hall

,playerID,N_all_star_games,inducted
0,aaronha01,24,Y
2,musiast01,24,Y
1,mayswi01,24,Y
3,mantlmi01,19,Y
4,ripkeca01,19,Y
...,...,...,...
1328,saundmi01,1,N
1329,bannifl01,1,N
1330,scarbra01,1,N
1331,scheiri01,1,N




Making a table `missing_hall_of_famers`. It shows the 10 players who played in the most all star games who were none the less NOT inducted into the hall of fame. 

In [27]:
# missing_hall_of_famers = all_stars_vs_hall["inducted"]
# missing_hall_of_famers = all_stars_vs_hall.DataFrame(all_stars_vs_hall["inducted"] = 'N')
# Selecting the product of HomeAppliances Type
missing_hall_of_famers = all_stars_vs_hall.loc[all_stars_vs_hall['inducted'] == 'N']
missing_hall_of_famers.head(10)

,playerID,N_all_star_games,inducted
11,rosepe01,17,N
23,rodrial01,14,N
19,bondsba01,14,N
17,jeterde01,14,N
26,riverma01,13,N
37,mcgwima01,12,N
38,ramirma02,12,N
47,clemero02,11,N
48,howarel01,11,N
50,crandde01,11,N


In [28]:
named_snubs = pd.read_sql_query(your_query2, con)

named_snubs = pd.merge(missing_hall_of_famers, names, on='playerID').drop_duplicates(subset=None, keep='first', inplace=False)
named_snubs

,playerID,N_all_star_games,inducted,nameLast,nameFirst
0,rosepe01,17,N,Rose,Pete
17,rodrial01,14,N,Rodriguez,Alex
31,bondsba01,14,N,Bonds,Barry
45,jeterde01,14,N,Jeter,Derek
59,riverma01,13,N,Rivera,Mariano
...,...,...,...,...,...
4050,saundmi01,1,N,Saunders,Michael
4051,bannifl01,1,N,Bannister,Floyd
4052,scarbra01,1,N,Scarborough,Ray
4053,scheiri01,1,N,Scheinblum,Richie


## Biggests snubs



1) **Pete Rose** is a highly decorated baseball athlete with multiple championships, all star appearances, and even a MVP award. According to wikipedia, "He's also credited with popularizing the head-first sliding technique in the majors." Unfortunately, Petes hall of fame eligibility was taken away because of gambling accusations. I believe that given his influence and accomplishments, this is the only reason he has not been accepted into the hall of fame.

2) like Pete, **Alex Rodriguez** is an extremely talented baseball player with numerous accolates and and influence. Alex started his career as a very promising young prospect and according to some, had potential to be the best baseball player ever. However, because of external pressures, alex has admitted to using performance enchancing drugs during his career which is likely the reason why he is ineligible for the hall of fame. 